# Cars example

as [requested by zhangliye](https://discourse.julialang.org/t/simulate-v0-2-0-a-julia-package-for-discrete-event-simulation/31822/9).  Zhangliye pointed out that there is no explanation of how to implement shared resources in `Simulate.jl`. The [`SimPy example`](https://simpy.readthedocs.io/en/latest/simpy_intro/shared_resources.html) cited by him simulates five cars, sharing a battery charging station with two charging points.

Resources can directly expressed in native Julia as tokens in a `Channel`.

In [1]:
using Simulate, Printf
struct Cable end           # this stands for a resource

Often in simulations resources hold some additional information. In those cases simply extend the struct.

In [2]:
function car(env::Clock, name::String, bcs::Channel, driving_time::Int, charge_duration::Int)
    delay!(env, driving_time)
    now!(env, SF(println, @sprintf("%s arriving at %d", name, tau(env))))
    cable = take!(bcs)                          # take the cable from the charging station
    now!(env, SF(println, @sprintf("%s starting to charge at %d", name, tau(env))))
    delay!(env, charge_duration)
    put!(bcs, cable)                            # put it back
    now!(env, SF(println, @sprintf("%s leaving the bcs at %d", name, tau(env))))
end

car (generic function with 1 method)

Note that the `now!`-call transfers the print operation to the clock and that we have to `put!` back explicitly the resource to the bcs after having used it (the SymPy example uses a `with` expression). 

In [3]:
env = Clock()
bcs = Channel{Cable}(2)    # define the battery charging station
for i in 1:2
    put!(bcs, Cable())     # with two cables (resources)
end
for i in 0:3
    process!(env, SP(i, car, env, "Car $i", bcs, i*2, 5), 1)
end
run!(env, 20)

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12


"run! finished with 20 clock events, 0 sample steps, simulation time: 20.0"

Everything is pretty much the same as in the SimPy-example.